# Importing libraries

In [2]:
import pandas as pd                       #reading, writing and manipulating the data (using tables)
import numpy as np                        #Library for linear algebra and some probabiltity (raw data)
import tensorflow as tf                       
from keras.models import Sequential       #To create the sequential layer

#from keras.layers.core import Flatten, Dense, Dropout     #To create the model
#from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D  #To create the model
from keras.preprocessing import image             #used for image classification
from keras.preprocessing.image import ImageDataGenerator  #used to expand the training dataset in order to improve the performance and ability of the model to generalize

from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, Dropout
from keras.optimizers import SGD, Adam          #To use the optimizer
from keras.utils import np_utils  

#import cv2                                #CV (computer vision)
#from google.colab import files            #To be able to upload files

In [9]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU')]

In [10]:
tf.test.is_built_with_cuda()

True

# Uploading Dataset

The Data set contains 48 X 48-pixel grayscale images of the face. There are seven categories **(0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral)** present in the data. The CSV file contains two columns that are emotion that contains numeric code from 0-6 and a pixel column that includes a string surrounded in quotes for each image.

In [3]:
print('Upload the CSV file')
#uploaded = files.upload()
emotion_data = pd.read_csv('fer2013.csv')
emotion_data

Upload the CSV file


,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training
...,...,...,...
35882,6,50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...,PrivateTest
35883,3,178 174 172 173 181 188 191 194 196 199 200 20...,PrivateTest
35884,0,17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...,PrivateTest
35885,3,30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...,PrivateTest


#Storing Dataset

We will create different lists for storing **the labels and the pixels** of both the **training and testing data**. We check the usuage colume in the csv file for each row if it is training then the pixels will be appended to the training array, but if its testing then the pixels will be appended to the testing array.

In [4]:
#---------------------- Spliting the DataSet------------------------------------
X_train = []      #train pixels
y_train = []      #train labels
X_test = []       #test pixels
y_test = []       #test label
for index, row in emotion_data.iterrows():    #iterate on each row and store it
    k = row['pixels'].split(" ")       #list of pixels, each pixel is an element
    try:
      if row['Usage'] == 'Training':    
          X_train.append(np.array(k,'float32'))    #The desired data-type for the array is 
          y_train.append(row['emotion'])
      elif row['Usage'] == 'PublicTest':
          X_test.append(np.array(k,'float32'))
          y_test.append(row['emotion'])
    except:
      print(f"error occured at index :{index} and row:{row}")





print(type(X_test))



print(type(y_train))

<class 'list'>
<class 'list'>




1.   Convert the arrays to numpy arrays so we can do several operations on it related to numpy library only like reshaping and dividing into categories
2.   Reshape the pixels arrays to be 48*48 array and each array has 1 element/pixel
3.   Convert the labels arrays into categorial ones, this function returns a matrix of binary values and have **rows numbers = the input vector dimensions** and have **columns numbers = the number of categories**  
        
        For example:


```
#Function
train_labels = to_categorical(train_labels, num_classes = 10)

#train_lables array before the function
array([[6],[9],[9]) 

#Labels after applying the function with 10 categories
[[0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 1]]

```



In [5]:
#--------------------Convert Lists to Numpy arrays------------------------------
X_train = np.array(X_train, 'float32')
y_train = np.array(y_train, 'float32')
X_test = np.array(X_test,'float32')
y_test = np.array(y_test,'float32')

#-----------------------Reshape Pixels arrays---------------------------------
#normalizing data between o and 1  
X_train -= np.mean(X_train, axis=0)  
X_train /= np.std(X_train, axis=0)  

X_test -= np.mean(X_test, axis=0)  
X_test /= np.std(X_test, axis=0) 


#reshape the numpy array to be passed to the model(Making the actual size of the image)
X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)   
X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

print(X_test.shape)
print(type(X_test))
print(X_train.shape)

#------------------Convert Labels array to categorial ones---------------------
#y_train= tf.keras.utils.to_categorical(y_train, num_classes=7)
#y_test = tf.keras.utils.to_categorical(y_test, num_classes=7)

y_train= np_utils.to_categorical(y_train, num_classes=7)
y_test = np_utils.to_categorical(y_test, num_classes=7)
print(y_train)
print(y_train.shape)
print(type(y_train))



(3589, 48, 48, 1)
<class 'numpy.ndarray'>
(28709, 48, 48, 1)
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]
(28709, 7)
<class 'numpy.ndarray'>


# Build CNN model

We will be building a specific type of CNN model which is the VGG16 (Very Deep Convolutional Networks for Large-Scale Image Recognition) which is Convolutional Network for Classification and Detection.
The model will consist of initialization of the model followed by batch normalization layer and then different convents layers with ReLu as an activation function, max pool layers, and dropouts to do learning efficiently.

In [ ]:
#model1 = Sequential()

#model1.add(ZeroPadding2D((1,1),input_shape=(48,48,1)))
#model1.add(Convolution2D(64, 3, 3, activation='relu'))
#model1.add(ZeroPadding2D((1,1)))
#model1.add(Convolution2D(64, 3, 3, activation='relu'))
#model1.add(MaxPooling2D(pool_size=(2,2), strides=(2,2),padding="same"))

#model1.add(ZeroPadding2D((1,1)))
#model1.add(Convolution2D(128, 3, 3, activation='relu'))
#model1.add(ZeroPadding2D((1,1)))
#model1.add(Convolution2D(128, 3, 3, activation='relu'))
#model1.add(MaxPooling2D(pool_size=(2,2), strides=(2,2),padding="same"))

#model1.add(ZeroPadding2D((1,1)))
#model1.add(Convolution2D(256, 3, 3, activation='relu'))
#model1.add(ZeroPadding2D((1,1)))
#model1.add(Convolution2D(256, 3, 3, activation='relu'))
#model1.add(ZeroPadding2D((1,1)))
#model1.add(Convolution2D(256, 3, 3, activation='relu'))
#model1.add(MaxPooling2D(pool_size=(2,2), strides=(2,2),padding="same"))

#model1.add(ZeroPadding2D((1,1)))
#model1.add(Convolution2D(512, 3, 3, activation='relu'))
#model1.add(ZeroPadding2D((1,1)))
#model1.add(Convolution2D(512, 3, 3, activation='relu'))
#model1.add(ZeroPadding2D((1,1)))
#model1.add(Convolution2D(512, 3, 3, activation='relu'))
#model1.add(MaxPooling2D(pool_size=(2,2), strides=(2,2),padding="same"))


#model1.add(ZeroPadding2D((1,1)))
#model1.add(Convolution2D(512, 3, 3, activation='relu'))
#model1.add(ZeroPadding2D((1,1)))
#model1.add(Convolution2D(512, 3, 3, activation='relu'))
#model1.add(ZeroPadding2D((1,1)))
#model1.add(Convolution2D(512, 3, 3, activation='relu'))
#model1.add(MaxPooling2D((2,2), strides=(2,2),padding="same"))

#model1.add(Flatten())
#model1.add(Dense(4096, activation='relu'))
#model1.add(Dropout(0.5))
#model1.add(Dense(4096, activation='relu'))
#model1.add(Dropout(0.5))
#model1.add(Dense(7, activation='softmax'))


#model1.summary()

In [6]:
model = Sequential()

model.add(Conv2D(input_shape=(X_train.shape[1:]),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))


model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))


model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 64)        36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 12, 12, 256)       2

# Compile Model

We compile the model using Adams as optimizer,loss as categorical cross-entropy, and metrics as accuracy

In [11]:
model.compile(optimizer=Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])

We fit the data for validation and training. Then take the batch size = 32 and epoch size = 30 

In [12]:
batch = 32
epoch = 30

history = model.fit(X_train,y_train,batch_size= batch,epochs= epoch,verbose=1,validation_data=(X_test, y_test),shuffle=True)



Epoch 1/30
898/898 [==============================] - 1470s 2s/step - loss: 1.6543 - accuracy: 0.3284 - val_loss: 1.4555 - val_accuracy: 0.4244
Epoch 2/30
280/898 [========>.....................] - ETA: 21:45 - loss: 1.4378 - accuracy: 0.4278

KeyboardInterrupt: 

#Evaluate the model

In [ ]:
loss_and_metrics = model.evaluate(X_test,y_test)
print(loss_and_metrics)


113/113 [==============================] - 1s 12ms/step - loss: 2.6882 - accuracy: 0.5946
[2.688185214996338, 0.5945945978164673]


#Save the model

We will serialize the model to JSON and save the model's weights in a hd5 file, so we can use it to make predictions directly without retraining the model everytime.

In [ ]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
  json_file.write(model_json)
  model.save_weights("model.h5")

print("Saved model to disk")

Saved model to disk


The real time detection will be done using Jupiter notebook and the JSON file with the model and the weights. 

#Test Model

In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from keras.models import model_from_json
from keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


In [3]:
model = model_from_json(open("models.json", "r").read())
model.load_weights('model.h5') 
face_haar_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')    

  #select the default video capture|

#If the camera was not opened sucessfully

In [4]:
cap=cv2.VideoCapture(0)
cap.set(3,1280)
cap.set(4,720)
cap.set(10,70)
if not cap.isOpened():  
    print("Cannot open camera")
    exit()
    
#Continously read the frames 
while True:
    #read frame by frame and get return whether there is a stream or not
    ret, frame=cap.read()
    
    #If no frames recieved, then break from the loop
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
        
    #Change the frame to greyscale  
    gray_image= cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #We pass the image, scaleFactor and minneighbour
    faces_detected = face_haar_cascade.detectMultiScale(gray_image,1.32,5)
    
    #Draw Triangles around the faces detected
    for (x,y,w,h) in faces_detected:
        cv2.rectangle(frame,(x,y), (x+w,y+h), (255,0,0), thickness=7)
        roi_gray=gray_image[y:y+w,x:x+h]
        roi_gray=cv2.resize(roi_gray,(48,48))
        
        #Processes the image and adjust it to pass it to the model
        image_pixels = tf.keras.preprocessing.image.img_to_array(roi_gray)
#         plt.imshow(image_pixels)
#         plt.show()
        image_pixels = np.expand_dims(image_pixels, axis = 0)
        image_pixels /= 255

        #Get the prediction of the model
        predictions = model.predict(image_pixels)
        print(predictions)
        max_index = np.argmax(predictions[0])
        emotion_detection = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
        emotion_prediction = emotion_detection[max_index]
        
        print(emotion_prediction)
        
        #Write on the frame the emotion detected
        cv2.putText(frame,emotion_prediction,(int(x), int(y)),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
    
    
    resize_image = cv2.resize(frame, (1000, 700))
    cv2.imshow('Emotion',resize_image)
    if cv2.waitKey(10) == ord('b'):
            break


cap.release()


[[1.8049935e-02 1.0746281e-05 9.3849823e-02 1.0113819e-03 4.4803768e-01
  2.7364847e-04 4.3876669e-01]]
sad
[[1.3780739e-02 3.2490007e-05 6.0321236e-01 1.0047740e-03 2.3365700e-01
  1.3122319e-03 1.4700040e-01]]
fear
[[8.9035500e-03 1.4232132e-05 6.7527950e-01 4.2528339e-04 2.5017515e-01
  5.5618508e-04 6.4646117e-02]]
fear
[[8.9811478e-03 9.9957924e-06 6.1556786e-01 3.8014285e-04 3.2899264e-01
  3.1948014e-04 4.5748748e-02]]
fear
[[8.1037227e-03 1.7175214e-05 8.4428030e-01 3.2785349e-04 9.7211495e-02
  9.4811525e-04 4.9111359e-02]]
fear
[[9.0201618e-03 1.3924993e-05 7.0369619e-01 4.3704675e-04 2.6941451e-01
  3.3114591e-04 1.7087054e-02]]
fear
[[7.1138320e-03 1.3515808e-05 8.5395670e-01 2.8408461e-04 1.1289499e-01
  6.1125722e-04 2.5125675e-02]]
fear
[[2.3206912e-02 5.3581851e-05 6.6150945e-01 1.3845027e-03 2.7760336e-01
  8.8288944e-04 3.5359297e-02]]
fear
[[1.4705143e-02 3.3821012e-05 6.5291947e-01 6.1380491e-04 3.6735758e-02
  3.9765397e-03 2.9101548e-01]]
fear
[[1.2140796e-02 3.18

[[4.5652483e-02 1.0180701e-05 2.2650944e-02 1.1396860e-03 8.8841915e-02
  2.5442190e-04 8.4145045e-01]]
neutral
[[5.3904451e-02 3.1183306e-05 5.7928700e-02 1.8399835e-03 1.9908319e-01
  5.1947549e-04 6.8669295e-01]]
neutral
[[5.3745937e-02 3.3040327e-05 6.2309217e-02 1.7704655e-03 1.3690300e-01
  6.7005388e-04 7.4456829e-01]]
neutral
[[7.1355134e-02 3.3298656e-05 3.4872539e-02 2.3325656e-03 2.5895265e-01
  3.4054698e-04 6.3211328e-01]]
neutral
[[7.0080750e-02 2.1209202e-05 2.7562112e-02 1.6126974e-03 1.2592423e-01
  3.1434602e-04 7.7448463e-01]]
neutral
[[0.27760318 0.00041046 0.12108088 0.00436623 0.24996859 0.00126988
  0.3453008 ]]
neutral
[[1.0629791e-01 1.7791575e-05 1.3823824e-02 1.3596026e-03 4.4833954e-02
  2.5320047e-04 8.3341378e-01]]
neutral
[[1.4641272e-01 5.3335400e-05 2.7836271e-02 2.4108337e-03 6.2695481e-02
  5.8289507e-04 7.6000845e-01]]
neutral
[[1.7490707e-01 8.3770276e-05 2.9241210e-02 3.2011494e-03 1.5953077e-01
  4.6835595e-04 6.3256770e-01]]
neutral
[[9.4367318e-

[[1.6790019e-02 2.0197378e-05 9.7685713e-01 7.2846982e-05 5.9210658e-03
  1.9479438e-04 1.4394337e-04]]
fear
[[5.4578770e-02 1.0097693e-04 9.1871488e-01 3.0748465e-04 2.3806162e-02
  5.9662160e-04 1.8951350e-03]]
fear
[[2.3806840e-02 7.7537416e-06 3.8082950e-02 1.0211657e-03 8.5073821e-02
  3.8276811e-04 8.5162467e-01]]
neutral
[[0.40949523 0.00049441 0.16721466 0.00413337 0.14498167 0.00163296
  0.27204773]]
angry
[[7.9413879e-01 8.7298499e-04 4.2047214e-02 3.7563168e-03 6.8525784e-02
  6.8334414e-04 8.9975588e-02]]
angry
[[5.9216058e-01 2.1579632e-04 2.7589105e-02 2.8830275e-03 4.3997526e-02
  5.6545617e-04 3.3258855e-01]]
angry
[[2.5379723e-01 6.4279819e-05 1.5434971e-02 2.3822852e-03 5.3164318e-02
  3.6702404e-04 6.7478997e-01]]
neutral
[[5.31398412e-03 4.04916108e-07 5.43788541e-03 4.40183998e-04
  1.14982827e-02 1.16807685e-04 9.77192342e-01]]
neutral
[[1.1677923e-01 4.5758650e-05 1.7977925e-02 3.0895192e-03 1.1099121e-01
  4.1001732e-04 7.5070632e-01]]
neutral
[[3.7704408e-01 4.

[[9.2419602e-02 1.2012081e-04 3.3976841e-01 1.7566165e-03 4.2281422e-01
  7.1503228e-04 1.4240602e-01]]
sad
[[1.2914431e-01 6.2393665e-05 3.6431834e-02 2.1204441e-03 7.8368092e-01
  7.1127215e-05 4.8489045e-02]]
sad
[[8.6898185e-02 5.1054729e-05 5.5320047e-02 1.7859382e-03 8.1098998e-01
  8.2422157e-05 4.4872303e-02]]
sad
[[3.2127928e-02 5.3573231e-06 8.7012229e-03 1.6397599e-03 8.3739763e-01
  2.3615445e-05 1.2010443e-01]]
sad
[[3.2320935e-02 4.7842568e-06 8.4548164e-03 1.2854808e-03 8.9817697e-01
  1.4190961e-05 5.9742793e-02]]
sad
[[1.2375712e-02 1.2856707e-06 8.2197050e-03 8.4597809e-04 8.1708103e-01
  1.4448753e-05 1.6146195e-01]]
sad
[[1.5846835e-02 1.9643626e-06 8.0020595e-03 1.1658866e-03 8.1303918e-01
  1.8359751e-05 1.6192578e-01]]
sad
[[1.7224781e-02 3.8557687e-06 1.7026221e-02 1.3459765e-03 5.8353311e-01
  6.5387387e-05 3.8080063e-01]]
sad
[[1.5586865e-02 1.3840436e-06 8.7274304e-03 8.2184799e-04 1.3215141e-01
  6.7397530e-05 8.4264368e-01]]
neutral
[[8.5103504e-02 2.505982

KeyboardInterrupt: 

In [5]:
success,img = cap.read()
    classIds, confs, bbox = net.detect(img,confThreshold=thres)
    print(classIds,bbox)
    print(f"shape of bbox {bbox.shape}")
    print(f"length of the classNames list is{len(classNames)}")

    if len(classIds) != 0:
        for classId, confidence,box in zip(classIds.flatten(),confs.flatten(),bbox):
            cv2.rectangle(img,box,color=(0,255,0),thickness=2)
            # print(f"box indexs is {box[0]} and {box[1]}")
            cv2.putText(img,classNames[classId-1].upper(),(box[0]+20,box[1]+20),
                        cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
            #         cv2.putText(img,str(round(confidence*100,2)),(box[0]+200,box[1]+30),
            #         cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)

    cv2.imshow("Output",img)
    cv2.waitKey(1)

IndentationError: unexpected indent (<ipython-input-5-a6de02657e5e>, line 2)